**Using the model trained by *CNN_train_without_metadata* I classified all the given 3469 test images into one of 10 classes and gave me predictions in a csv named CNN_Classification in this notebook**

Importing all necessary libraries

In [16]:
import os
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from PIL import Image
import glob

Checking if CUDA is avilable

In [17]:
print(f'torch version = {torch.__version__}')
#checking if CUDA is available
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device = {device}')

torch version = 2.5.0+cu118
device = cuda


Setting seed for reproducability

In [18]:
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Setting image and batch size

In [19]:
IMG_SIZE = 224
BATCH_SIZE = 32

Defining test data path and test tranforms

In [20]:
DATASET_PATH = '.'  # Main dataset directory
TRAIN_PATH = os.path.join(DATASET_PATH, 'train_images')
TEST_PATH = os.path.join(DATASET_PATH, 'test_images')

test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Defining class to prepare dataset

In [21]:
class PaddyLeafDatasetWithoutMetadata(Dataset):
    def __init__(self, root_dir,transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.samples = []
        for img_path in glob.glob(os.path.join(self.root_dir, '*.jpg')):
            self.samples.append((img_path))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path = self.samples[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image ,img_path

Made test_dataset

In [22]:
# Load test dataset (without metadata)
test_dataset = PaddyLeafDatasetWithoutMetadata(
    TEST_PATH,
    transform=test_transforms,
)

test_loader=DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)

In [23]:
print(len(test_dataset))

3469


Defined Model architecture 

In [24]:
class MetadataAwareModel(nn.Module):
    def __init__(self, base_model_name='resnet50', num_classes=10, num_varieties=1):
        super(MetadataAwareModel, self).__init__()

        # Initialize the CNN backbone
        if base_model_name == 'resnet50':
            self.backbone = models.resnet50(pretrained=True)
            self.feature_dim = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()  # Remove the final fully connected layer

        elif base_model_name == 'efficientnet':
            self.backbone = models.efficientnet_b0(pretrained=True)
            self.feature_dim = self.backbone.classifier[1].in_features
            self.backbone.classifier = nn.Identity()  # Remove the final classifier

        elif base_model_name == 'densenet':
            self.backbone = models.densenet121(pretrained=True)
            self.feature_dim = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()  # Remove the final classifier

        # Layers for processing metadata
        self.variety_fc = nn.Linear(num_varieties, 64)
        self.age_fc = nn.Linear(1, 16)

        # Combined feature dimension
        combined_dim = self.feature_dim + 64 + 16

        # Final classifier
        self.classifier = nn.Sequential(
            nn.Linear(combined_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, image, variety, age):
        # Process image through the backbone
        image_features = self.backbone(image)

        # Process metadata
        variety_features = torch.relu(self.variety_fc(variety))
        age_features = torch.relu(self.age_fc(age))

        # Concatenate all features
        combined_features = torch.cat([image_features, variety_features, age_features], dim=1)

        # Final classification
        output = self.classifier(combined_features)

        return output
    


class ImageOnlyModel(nn.Module):
    def __init__(self, pretrained_model_path, base_model_name='densenet', num_classes=10):
        super(ImageOnlyModel, self).__init__()

        # Load the pre-trained model
        pretrained_model = MetadataAwareModel(base_model_name=base_model_name, num_classes=num_classes, num_varieties=10)
        pretrained_model.load_state_dict(torch.load(pretrained_model_path, map_location=device,weights_only=True))
        
        # Extract the CNN backbone from the pre-trained model
        self.backbone = pretrained_model.backbone  # Keeping the pretrained feature extractor
        self.feature_dim = pretrained_model.feature_dim  # Feature size from CNN

        # New classifier (without metadata features)
        self.classifier = nn.Sequential(
            nn.Linear(self.feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, image):
        # Extract image features
        image_features = self.backbone(image)

        # Pass through new classifier
        output = self.classifier(image_features)

        return output

Created model

In [26]:
model_name = 'densenet'  # Choose from: 'resnet50', 'efficientnet', 'densenet'
model=ImageOnlyModel('paddy_disease_densenet_with_metadata.pth',base_model_name=model_name,num_classes=10)
model = model.to(device)

Loaded weights from .pth files

In [27]:
checkpoint_path = "paddy_disease_densenet_without_metadata.pth"
model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),weights_only=True))

<All keys matched successfully>

Defined a model which stores predictions in a DataFrame

In [28]:
def prediction_model(model, dataloader):
    model.eval()
    all_preds = []
    image_names = []  # To store image names

    with torch.no_grad():
        for inputs, paths in dataloader:
          inputs = inputs.to(device)

          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)

          all_preds.extend(preds.cpu().numpy())
          image_names.extend(paths)  # Collect image names

    # Store predictions in a DataFrame
    df = pd.DataFrame({
        "image_id": [os.path.basename(p) for p in image_names],  # Extract only filename
        "label": all_preds
    })
    return df

Doing predictions on test data

In [29]:
predictions_df = prediction_model(model, test_loader)

Prepared an encoding to class mapping

In [30]:
class_to_idx={'bacterial_leaf_blight': 0, 'bacterial_leaf_streak': 1, 'bacterial_panicle_blight': 2, 'blast': 3, 'brown_spot': 4, 'dead_heart': 5, 'downy_mildew': 6, 'hispa': 7, 'normal': 8, 'tungro': 9}
idx_to_class = {v: k for k, v in class_to_idx.items()}

print(idx_to_class)

{0: 'bacterial_leaf_blight', 1: 'bacterial_leaf_streak', 2: 'bacterial_panicle_blight', 3: 'blast', 4: 'brown_spot', 5: 'dead_heart', 6: 'downy_mildew', 7: 'hispa', 8: 'normal', 9: 'tungro'}


Created a CSV with columns -> image_id and label , which is sorted according to image_id

In [ ]:
for i in range(len(predictions_df)):
  predictions_df['label'][i]=idx_to_class[predictions_df['label'][i]]

df_sorted = predictions_df.sort_values(by="image_id")

# Save the sorted DataFrame to a new CSV file
df_sorted.to_csv("CNN_Classification.csv", index=False)

/tmp/ipykernel_2861202/1122322357.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  predictions_df['label'][i]=idx_to_class[predictions_df['label'][i]]
/tmp/ipykernel_2861202/1122322357.py:2: SettingWithCopyWarning: 
A value is trying to b

: 